<a href="https://colab.research.google.com/github/vincm1/RecSys_Implicit/blob/master/Bayesian_Personalized_Ranking_(BPR).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bayesian Personalized Ranking

This notebook will use the pairwise-ranking algorithm BPR to recommend and rank Top10 items based on the paper of Rendle et al. 2009 (https://arxiv.org/ftp/arxiv/papers/1205/1205.2618.pdf)

Therefore two different RecSys libraries will be used:



*   Implicit BPR(https://benfred.github.io/implicit/ by Ben Frederickson)

In [1]:
import warnings
import zipfile
import time
import pickle
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

import scipy.sparse as sparse

from datetime import datetime, timedelta
warnings.filterwarnings("ignore")

## Data Preprocessing

In [2]:
df_zip = zipfile.ZipFile('/content/drive/MyDrive/RecSys/Orders_Nov22_Jun23.zip')
df = pd.read_csv(df_zip.open('Bericht 1.csv'), delimiter=";")

In [3]:
df.head(2)

,Order Nbr,Entry Date,Entry DateTime,FiscalMonth,BranchCustomerNbr,CustomerName,BusinessUnitLevel2,KDGroup,Sku,Product Descr1,Product Descr2,ProductGroup,ProductGroupMasterDescription,ProductGroupDescription,ProductGroup2ndDescription,Sales,Qty Shipped
0,1547606,01.11.22,NaN,2022FM11,15515778,NET-S M. CHMIELEWSKI,Export Channel (DE),,9433B9X,INK CARTRIDGE SPS,BLACK 370ML 600 DPI INKJET BULK,1037,Consumables,Ink,Supplies,"-15,33",-1.0
1,1547615,01.11.22,NaN,2022FM11,15509465,DIGITAL RIVER IRELAND LIMITED,Export Channel (DE),DIRL,CB31510,LENOVO KEYBOARD PACK,FOR TAB P11-DE,641,Input Devices,Keyboards & Keypads,Printers & Peripherals,"-104,61",-1.0


In [4]:
print(len(df), len(df["BranchCustomerNbr"].unique()), len(df["Sku"].unique()))

2220299 17697 77401


In [5]:
# converting the customerid to string
df["BranchCustomerNbr"] = df["BranchCustomerNbr"].astype(str)
# converting the skuid to string
df["Sku"] = df["Sku"].astype(str)
# Entry Date to date
df['Entry Date'] = pd.to_datetime(df['Entry Date'], format='%d.%m.%y')
# dropping retours (orders with negative Qty shipped) and zero Qty shipped orders
df = df[df["Qty Shipped"] > 0]
# dropping backlog invoices, Specified date to filter the rows
specific_date = pd.to_datetime('2022-11-01')
# Filter the DataFrame to keep only the rows that are before or equal to the specific date
df = df[df["Entry Date"] >= specific_date]
#insert purchase indication column
df["purchase"] = 1

In [6]:
print(len(df), len(df["BranchCustomerNbr"].unique()), len(df["Sku"].unique()))

2028956 13894 75643


In [7]:
# add column that represent sku count
def sku_count(df_pl):

  df_pl['sku_count'] = df_pl.groupby('BranchCustomerNbr')['Sku'].transform('nunique')

  return df_pl

In [8]:
df = sku_count(df)

In [9]:
print(len(df), len(df["BranchCustomerNbr"].unique()), len(df["Sku"].unique()))

2028956 13894 75643


In [10]:
# drop users
df = df[df.sku_count > 1]

In [11]:
# Create a numeric user_id and artist_id column
df['BranchCustomerNbr'] = df['BranchCustomerNbr'].astype("category")
df['Sku'] = df['Sku'].astype("category")
df['bcn_id'] = df['BranchCustomerNbr'].cat.codes
df['sku_id'] = df['Sku'].cat.codes

In [12]:
df.head(2)

,Order Nbr,Entry Date,Entry DateTime,FiscalMonth,BranchCustomerNbr,CustomerName,BusinessUnitLevel2,KDGroup,Sku,Product Descr1,...,ProductGroup,ProductGroupMasterDescription,ProductGroupDescription,ProductGroup2ndDescription,Sales,Qty Shipped,purchase,sku_count,bcn_id,sku_id
213,1545306,2022-11-02,NaN,2022FM11,15885514,AXIS SOLUTION (PRIVATE) LIMITED,Export Channel (DE),,CF55877,DT PRINT ZQ320 KIT LABEL SENSOR,...,5805,AIDC/PoS Printers,Mobile Receipt Printer,"Other (incl. AIDC/POS, V7)","10.713,30",41.0,1,20,417,49730
458,4422886,2022-11-03,NaN,2023FM02,44413224,BWG INFORMATIONSYSTEME GMBH,Business Channel,,CF89211,Z-SELECT 2000D REMOVABLE NS,...,5812,AIDC/PoS Printers,Label Printers Supplies,"Other (incl. AIDC/POS, V7)","1.393,00",140.0,1,111,803,52649


In [13]:
users = df.bcn_id.unique()
skus = df.sku_id.unique()
print(len(users), len(skus))

11328 75495


In [14]:
df_2 = df[["bcn_id", "sku_id", "Entry Date", "Qty Shipped", "purchase"]]

In [15]:
grouped_df = df_2.groupby(["bcn_id", "sku_id"]).agg({
          "Entry Date":"max",
          "Qty Shipped":"sum",
          "purchase":"sum"}).reset_index()

In [16]:
# create binary column
grouped_df["purch_bin"] = 1

In [17]:
grouped_df

,bcn_id,sku_id,Entry Date,Qty Shipped,purchase,purch_bin
0,0,925,2023-02-03,1.0,1,1
1,0,3417,2023-03-31,1.0,1,1
2,0,3418,2023-06-26,4.0,2,1
3,0,3419,2023-04-17,3.0,1,1
4,0,4184,2023-01-25,4.0,1,1
...,...,...,...,...,...,...
541846,11326,33694,2023-01-11,9.0,1,1
541847,11326,42224,2023-02-24,2.0,1,1
541848,11326,45153,2023-01-11,2.0,1,1
541849,11327,35982,2022-12-14,25.0,1,1


In [18]:
def train_test_split(df, holdout_num):
    """ perform training testing split

    @param df: dataframe
    @param holdhout_num: number of items to be held out per user as testing items

    @return df_train: training data
    @return df_test testing data

    """
    # first sort the data by time
    df = df.sort_values(['bcn_id', 'Entry Date'], ascending=[True, False])

    # perform deep copy to avoid modification on the original dataframe
    df_train = df.copy(deep=True)
    df_test = df.copy(deep=True)

    # get test set
    df_test = df_test.groupby(['bcn_id']).head(holdout_num).reset_index()

    # get train set
    df_train = df_train.merge(
        df_test[['bcn_id', 'sku_id']].assign(remove=1),
        how='left'
    ).query('remove != 1').drop('remove', 1).reset_index(drop=True)

    # Sanity check to make sure we're not duplicating/losing data
    assert len(df) == len(df_train) + len(df_test)

    return df_train, df_test

In [19]:
df_train, df_test = train_test_split(grouped_df[["bcn_id","sku_id", "Entry Date", "purch_bin"]], holdout_num=1)

In [20]:
set(df_test.bcn_id.unique()).issubset(set(df_train.bcn_id.unique()))

True

In [21]:
train_bcn_ids = set(df_train['bcn_id'].unique())

# Filter train DataFrame to include only bcn_ids present in the test set
df_test_filtered = df_test[df_test['bcn_id'].isin(set(df_train['bcn_id'].unique()))]
#df_test_filtered = df_test_filtered[df_test_filtered['sku_id'].isin(set(df_train['sku_id'].unique()))]

In [22]:
set(df_test_filtered.bcn_id.unique()).issubset(set(df_train.bcn_id.unique()))

True

## Negative Sampling

In [23]:
grouped_df_binary = grouped_df[["bcn_id", "sku_id", "purch_bin"]]

In [24]:
len(df_train)

530523

In [25]:
len(df_test_filtered)

11328

In [26]:
len(df_test_filtered) / (len(df_test_filtered) + len(df_train))

0.020906116257052215

In [27]:
def negative_sampling(bcn_ids, sku_ids, items, n_neg):
    """This function creates n_neg negative labels for every positive label

    @param user_ids: list of user ids
    @param movie_ids: list of movie ids
    @param items: unique list of movie ids
    @param n_neg: number of negative labels to sample

    @return df_neg: negative sample dataframe

    """

    neg = []
    ui_pairs = zip(bcn_ids, sku_ids)
    records = set(ui_pairs)

    # for every positive label case
    for (u, i) in records:
        # generate n_neg negative labels
        for _ in range(n_neg):
            j = np.random.choice(items)
            # resample if the movie already exists for that user
            while (u, j) in records:
                j = np.random.choice(items)
            neg.append([u, j, 0])

    # convert to pandas dataframe for concatenation later
    df_neg = pd.DataFrame(neg, columns=['bcn_id', 'sku_id', 'purch_bin'])

    return df_neg

In [28]:
df_neg = negative_sampling(df_train.bcn_id, df_train.sku_id, grouped_df.sku_id.unique(), 10)

In [29]:
df_train_sam = pd.concat([df_train[["bcn_id","sku_id","purch_bin"]], df_neg], ignore_index=True).sort_values(by="bcn_id", ascending=True)

## Implicit BPR

In [32]:
!pip install implicit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 58.9 MB/s eta 0:00:00


In [33]:
import implicit
from implicit.gpu.bpr import BayesianPersonalizedRanking as BPR
from implicit.evaluation import leave_k_out_split, precision_at_k, mean_average_precision_at_k, ndcg_at_k, AUC_at_k, train_test_split
from sklearn.model_selection import GridSearchCV

In [34]:
csr_train = sparse.csr_matrix((df_train_sam['purch_bin'], (df_train_sam['bcn_id'], df_train_sam['sku_id'])))
csr_test = sparse.csr_matrix((df_test_filtered['purch_bin'], (df_test_filtered['bcn_id'], df_test_filtered['sku_id'])))

### BINARY Based BPR

In [35]:
model = BPR(factors=200, regularization=0.01, learning_rate=0.01, iterations=20)
model.fit(csr_train)

  0%|          | 0/20 [00:00<?, ?it/s]

In [36]:
auc10 = AUC_at_k(model, csr_train, csr_test, K=10)
prec10 = precision_at_k(model, csr_train, csr_test, K=10)
ndcg10 = ndcg_at_k(model, csr_train, csr_test, K=10)
print(f"AUC@10: {auc10}; PREC@10: {prec10}; NDCG@10: {ndcg10}")

  0%|          | 0/11328 [00:00<?, ?it/s]

  0%|          | 0/11328 [00:00<?, ?it/s]

  0%|          | 0/11328 [00:00<?, ?it/s]

AUC@10: 0.5205038307215845; PREC@10: 0.04113700564971751; NDCG@10: 0.027702182509165233


## Hyperparam tuning

In [37]:
from IPython.utils.sysinfo import num_cpus
from sklearn.model_selection import ParameterGrid

# Grid of hyperparameters to search
param_grid = {
    'factors': [10, 50, 100, 150, 200],
    'iterations': [10, 50, 100, 200],
    'regularization': [0.01, 0.1],
    'learning_rate': [0.01, 0.1]
}

best_auc = -np.inf
best_params = {}

# Iterate through all parameter combinations
for params in ParameterGrid(param_grid):
    model = BPR(factors=params['factors'],
                    iterations=params['iterations'],
                    regularization=params['regularization'],
                    learning_rate=params['learning_rate'])
    model.fit(csr_train, show_progress=False)

    auc10 = AUC_at_k(model, csr_train, csr_test,
                    K=10, num_threads=10, show_progress=False)

    if auc10 > best_auc:
        best_auc = auc10
        best_params = params

# Print the best parameters and AUC
print("Best parameters:", best_params)
print("Best AUC:", best_auc)

Best parameters: {'factors': 100, 'iterations': 200, 'learning_rate': 0.01, 'regularization': 0.1}
Best AUC: 0.529199585585788


In [38]:
model = BPR(**best_params)
model.fit(csr_train)

  0%|          | 0/200 [00:00<?, ?it/s]

In [39]:
auc10 = AUC_at_k(model, csr_train, csr_test, K=10)
prec10 = precision_at_k(model, csr_train, csr_test, K=10)
map10 = mean_average_precision_at_k(model, csr_train, csr_test, K=10)
ndcg10 = ndcg_at_k(model, csr_train, csr_test, K=10)
print(f"AUC@10: {auc10}; PREC@10: {prec10}; MAP@10: {map10}; NDCG@10: {ndcg10}")

  0%|          | 0/11328 [00:00<?, ?it/s]

  0%|          | 0/11328 [00:00<?, ?it/s]

  0%|          | 0/11328 [00:00<?, ?it/s]

  0%|          | 0/11328 [00:00<?, ?it/s]

AUC@10: 0.528537578313771; PREC@10: 0.057203389830508475; MAP@10: 0.03097718253968251; NDCG@10: 0.03725039830606989


# Recommending

In [40]:
sku_list = df[["sku_id", "Product Descr1", "ProductGroupDescription", "ProductGroupMasterDescription"]].drop_duplicates()
sku_list.head(1)

,sku_id,Product Descr1,ProductGroupDescription,ProductGroupMasterDescription
213,49730,DT PRINT ZQ320 KIT LABEL SENSOR,Mobile Receipt Printer,AIDC/PoS Printers


In [41]:
# randomly selected userids on two clusters, see ALS repo
user_id_bcn = "44508633"
user_id_2_bcn = "44510285"

In [42]:
# find the bcn_id by bcn id
user_id= df.loc[df['BranchCustomerNbr'] == user_id_bcn, 'bcn_id'].head(1).values[0]
user_id_2 = df.loc[df['BranchCustomerNbr'] == user_id_2_bcn, 'bcn_id'].head(1).values[0]
print(user_id)
print(user_id_2)

5865
5990


In [44]:
from google.colab import files
# Now you can call the recommend function
userid = [user_id]
ids, scores = model.recommend(userid,csr_train[userid], N=10, filter_already_liked_items=True)
ids, scores

rec_tab = pd.DataFrame(data=[ids[0],scores[0]])
rec_tab = rec_tab.T.rename(columns={0:"sku_id", 1:"score", 2:"sku_id", 3:"score"})
rec_tab = rec_tab.merge(sku_list, on="sku_id", how="left")
rec_tab.to_csv(f'{user_id}_BPR_BIN_REC.csv')
files.download(f'{user_id}_BPR_BIN_REC.csv')
# rec_tab.sort_values(by="conf", ascending=False)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [45]:
# Now you can call the recommend function
userid = [user_id_2]
ids, scores = model.recommend(userid, csr_train[userid], N=10, filter_already_liked_items=True)
ids, scores

rec_tab = pd.DataFrame(data=[ids[0],scores[0]])
rec_tab = rec_tab.T.rename(columns={0:"sku_id", 1:"score", 2:"sku_id", 3:"score"})
rec_tab = rec_tab.merge(sku_list, on="sku_id", how="left")
rec_tab.to_csv(f'{user_id_2}_BPR_BIN_REC.csv')
files.download(f'{user_id_2}_BPR_BIN_REC.csv')
# rec_tab.sort_values(by="conf", ascending=False)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>